# Adding a new Dataset to GRETEL

In [20]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
print(module_path)

c:\Users\Roberto\Documents\Projects\GRETEL


In [17]:
from src.evaluation.evaluator_manager import EvaluatorManager

config_file_path =  module_path + '/examples/config/config_squares-triangles_trisqr_custom_oracle_dce.json'
output_file_path = module_path + '/examples/output/triangles_squares_custom_oracle/DCESearchExplainer/results_run-0.json'
output_folder = module_path + '/examples/output/'
stats_folder = module_path + '/examples/stats/'
datasets_folder = module_path + '/data/datasets/'

# Verifying that the paths are valid
(os.path.isfile(config_file_path), os.path.isfile(output_file_path), os.path.isdir(output_folder), os.path.isdir(stats_folder), 
os.path.isdir(datasets_folder))

(True, False, True, True, True)

## Creating the Squares-Triangles Dataset

The Squares-Triangles Dataset is a synthetic dataset generated on the fly that is composed by cycle graphs, some of them are triangles and the others are squares

### Creating the dataset

In [18]:
from src.dataset.data_instance_base import DataInstance
from src.dataset.dataset_base import Dataset

import networkx as nx
import numpy as np

class SquaresTrianglesDataset(Dataset):

    def __init__(self, id, config_dict=None) -> None:
        super().__init__(id, config_dict)
        self.instances = []

    def create_cycle(self, cycle_size, role_label=1):

        # Creating an empty graph and adding the nodes
        graph = nx.Graph()
        graph.add_nodes_from(range(0, cycle_size))

        # Adding the edges  of the graph
        for i in range(cycle_size - 1):
            graph.add_edges_from([(i, i + 1)])

        graph.add_edges_from([(cycle_size - 1, 0)])
        
        # Creating the dictionary containing the node labels 
        node_labels = {}
        for n in graph.nodes:
            node_labels[n] = role_label

        # Creating the dictionary containing the edge labels
        edge_labels = {}
        for e in graph.edges:
            edge_labels[e] = role_label

        # Returning the cycle graph and the role labels
        return graph, node_labels, edge_labels


    def generate_squares_triangles_dataset(self, n_instances):

        self._name = ('squares-triangles_instances-'+ str(n_instances))

        # Creating the empty list of instances
        result = []

        for i in range(0, n_instances):
            # Randomly determine if the graph is going to be a traingle or a square
            is_triangle = np.random.randint(0,2)

            # Creating the instance
            data_instance = DataInstance(id=self._instance_id_counter)
            self._instance_id_counter +=1

            i_name = 'g' + str(i)
            i_graph = None
            i_node_labels = None
            i_edge_labels = None

            # creating the instance properties specific for squares or triangles
            if(is_triangle):
                # Creating the triangle graph
                i_graph, i_node_labels, i_edge_labels = self.create_cycle(cycle_size=3, role_label=1)
                data_instance.graph_label = 1
            else:
                i_graph, i_node_labels, i_edge_labels = self.create_cycle(cycle_size=4, role_label=0)
                data_instance.graph_label = 0  

            # Creating the general instance properties
            data_instance.graph = i_graph
            data_instance.node_labels = i_node_labels
            data_instance.edge_labels = i_edge_labels
            data_instance.minimum_counterfactual_distance = 4
            data_instance.name = i_name

            result.append(data_instance)

        # return the set of instances
        self.instances = result

    

### Creating the DatasetFactory

In [19]:
from src.dataset.dataset_base import Dataset
from src.dataset.dataset_factory import DatasetFactory
import os
import shutil


class CustomDatasetFactory(DatasetFactory):

    def __init__(self, data_store_path) -> None:
        self._data_store_path = data_store_path
        self._dataset_id_counter = 0


    def get_dataset_by_name(self, dataset_dict) -> Dataset:

        dataset_name = dataset_dict['name']
        params_dict = dataset_dict['parameters']

        # Check if the dataset is a squares-triangles dataset
        if dataset_name == 'squares-triangles':
            if not 'n_inst' in params_dict:
                raise ValueError('''"n_inst" parameter containing the number of instances in the dataset
                 is mandatory for squares-triangles dataset''')

            return self.get_squares_triangles_dataset(params_dict['n_inst'], False, dataset_dict)
        else:
            # call the base method in to generate any of the originally supported datasets
            return super().get_dataset_by_name(dataset_dict)


    def get_squares_triangles_dataset(self, n_instances=300, regenerate=False, config_dict=None) -> Dataset:
        result = SquaresTrianglesDataset(self._dataset_id_counter, config_dict)
        self._dataset_id_counter+=1

        # Create the name an uri of the dataset using the provided parameters
        ds_name = ('squares-triangles_instances-'+ str(n_instances))
        ds_uri = os.path.join(self._data_store_path, ds_name)
        ds_exists = os.path.exists(ds_uri)

        # If regenerate is true and the dataset exists then remove it an generate it again
        if regenerate and ds_exists: 
            shutil.rmtree(ds_uri)

        # Check if the dataset already exists
        if(ds_exists):
            # load the dataset
            result.read_data(ds_uri)
        else:
            # Generate the dataset
            result.generate_squares_triangles_dataset(n_instances)
            result.generate_splits()
            result.write_data(self._data_store_path)
            
        return result

### Evaluating the DCE explainer in the new dataset

In [21]:
from src.utils.context import Context
ds_store_path = datasets_folder
print(datasets_folder)
ds_factory = CustomDatasetFactory(ds_store_path)
context = Context.get_context(config_file=config_file_path)
# The run number is a way to differentiate many runs of the same configurations
eval_manager = EvaluatorManager(context)
eval_manager.create_evaluators()
eval_manager.evaluate()

c:\Users\Roberto\Documents\Projects\GRETEL/data/datasets/


KeyError: 'class'

### Reading the results

In [14]:
with open(output_file_path, 'r') as rs_json_reader:
                results = rs_json_reader.read()

results

'{"config": {"dataset": {"name": "squares-triangles", "parameters": {"n_inst": 100}}, "oracle": {"name": "trisqr_custom_oracle", "parameters": {"embedder": {"name": "graph2vec", "parameters": {}}}}, "explainer": {"name": "dce_search", "parameters": {"graph_distance": {"name": "graph_edit_distance", "parameters": {}}}}, "metrics": [{"name": "graph_edit_distance", "parameters": {}}, {"name": "oracle_calls", "parameters": {}}, {"name": "correctness", "parameters": {}}, {"name": "sparsity", "parameters": {}}, {"name": "fidelity", "parameters": {}}, {"name": "oracle_accuracy", "parameters": {}}]}, "runtime": [0.0008592605590820312, 0.0009090900421142578, 0.0008060932159423828, 0.0007355213165283203, 0.000823974609375, 0.0009407997131347656, 0.0024175643920898438, 0.0014560222625732422, 0.0011241436004638672, 0.0008006095886230469, 0.0006210803985595703, 0.0006079673767089844, 0.0006237030029296875, 0.0005993843078613281, 0.0007026195526123047, 0.0007009506225585938, 0.0006315708160400391, 0

Creating results table

In [15]:
from src.data_analysis.data_analyzer import DataAnalyzer

dtan = DataAnalyzer(output_folder, stats_folder)
dtan.aggregate_data()
dtan.aggregate_runs()
dtan.create_tables_by_oracle_dataset()

In [16]:
import pandas as pd
results_table = pd.read_csv(stats_folder + 'squares-triangles-trisqr_custom_oracle.csv')
results_table

,Unnamed: 0,explainer,runtime,runtime-std,Graph_Edit_Distance,Graph_Edit_Distance-std,Oracle_Calls,Oracle_Calls-std,Correctness,Correctness-std,Sparsity,Sparsity-std,Fidelity,Fidelity-std,Oracle_Accuracy,Oracle_Accuracy-std
0,0,dce_search,0.000808,0.0,4.0,0.0,101.0,0.0,1.0,0.0,0.578333,0.0,1.0,0.0,1.0,0.0
